##### Spoiler Alert! Spoiler Detection Project

## Train-Test-Split and Preprocessing 

In [1]:
reset -fs

In [2]:
import seaborn as sns
import pandas as pd
import numpy as np
import gzip
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from datetime import datetime

In [3]:
#Load datafile
df = pd.read_hdf('data/complete_data.h5')

### Train-Test-Split

In [4]:
# Split train, validation and test test with ratios 70% - 20% -10%
train, validation, test = np.split(df.sample(frac=1), [int(.7*len(df)), int(.9*len(df))])

In [5]:
#Save validation and test sets (train set will be saved after preprocessing) as HDF5
test.to_hdf('data/test_data.h5', key = 'test')
validation.to_hdf('data/validation_data.h5', key = 'validation')

/opt/anaconda3/envs/nf/lib/python3.6/site-packages/pandas/core/generic.py:2446: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->Index(['user_id', 'time', 'review', 'book_id', 'review_id', 'genres', 'title',
       'description', 'publication_year', 'publication_month',
       'publication_day', 'average_rating', 'ratings_count', 'num_pages'],
      dtype='object')]

  encoding=encoding,


### Data Preprocessing

From now on, only the train data is manipulated, validation and test sets are only worked on just before model evaluation.

In [6]:
#Reset the index
train = train.reset_index()

In [7]:
#Drop the feature containing the old index
train.drop('index', axis = 1, inplace = True)

In [8]:
#Information on data types and missing values
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 964623 entries, 0 to 964622
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   user_id            964623 non-null  object
 1   time               964623 non-null  object
 2   review             964623 non-null  object
 3   rating             964623 non-null  int64 
 4   spoiler            964623 non-null  bool  
 5   book_id            964623 non-null  object
 6   review_id          964623 non-null  object
 7   genres             964623 non-null  object
 8   title              964623 non-null  object
 9   description        964623 non-null  object
 10  publication_year   964623 non-null  object
 11  publication_month  964623 non-null  object
 12  publication_day    964623 non-null  object
 13  average_rating     964623 non-null  object
 14  ratings_count      964623 non-null  object
 15  num_pages          964623 non-null  object
dtypes: bool(1), int64(1)

In [9]:
#Show unique values in the sorted genres column. Since the genres column contains dictionaries, 
#the data type is temporarily changed to string format. 
values = train.genres.astype('str').sort_values(ascending = False)
values

379862    {'young-adult': 998, 'fantasy, paranormal': 13...
875606    {'young-adult': 998, 'fantasy, paranormal': 13...
956124    {'young-adult': 998, 'fantasy, paranormal': 13...
413489    {'young-adult': 998, 'fantasy, paranormal': 13...
811133    {'young-adult': 998, 'fantasy, paranormal': 13...
                                ...                        
646519                      {'children': 102, 'fiction': 7}
905797                      {'children': 102, 'fiction': 7}
406255                      {'children': 102, 'fiction': 7}
599548                      {'children': 102, 'fiction': 7}
696350                      {'children': 102, 'fiction': 7}
Name: genres, Length: 964623, dtype: object

#### Change data and data types

Obviously, some changes are necessary:
* Missing values are denoted as '' or '[]', respectively, and need to be changed to np.nan
* Datatypes need to be changed for some variables:
  * _time_ to date
  * _book_id_ to string
  * _publication_year_, _publication_month_, _publication_day_, _average_rating_, _ratings_count_, _num_pages_ to numeric

#### Change data types

In [10]:
#Change the data type of book_id to string
train.book_id = train.book_id.astype('str')

In [11]:
train.columns

Index(['user_id', 'time', 'review', 'rating', 'spoiler', 'book_id',
       'review_id', 'genres', 'title', 'description', 'publication_year',
       'publication_month', 'publication_day', 'average_rating',
       'ratings_count', 'num_pages'],
      dtype='object')

In [12]:
#Change the data type of time from object to date in the format (YYYY-MM-DD)
from datetime import datetime
train.time = pd.to_datetime(train.time)

In [13]:
#Change datatypes from object to floats.
to_num = ['average_rating', 'ratings_count', 'publication_year', 'publication_month', 'publication_day']
for col in to_num:
    train[col] = pd.to_numeric(train[col], errors = 'coerce')

#### Feature Engineering

We add a new feature containing the frequency-weighted average of book ratings.

In [14]:
train['weighted_avg_rating'] = train.average_rating * train.ratings_count

The genres column contains more than one genre assignment to the books. Since we only want one genre per book, we create a new column containing the genre most frequently allocated. 

In [15]:
#Define function fetching the most frequent (= value) genre (= key)
import operator

def get_genre(dic):
    
    ''' Return the key of the highest value of dictionary given in.
    If the dictionary is empty, return np.nan
    '''
    
    try:
        x = max(dic.items(), key = operator.itemgetter(1))[0]
        return x
    except:
        return np.nan  

In [16]:
#Use the function defined above to fetch the most frequent genre allocation.
#First, write all keys to a list.
genre = []
for i in range(len(train)):
    a = get_genre(train.genres[i])
    genre.append(a)

In [17]:
#Add the information from the list as a new column to the genre dataframe
train['genre'] = pd.Series(genre)

We compute another column with overall spoiler labels coded as 0 = "no spoiler" and 1 = "spoiler".

In [18]:
train['spoiler_dum'] = np.where(train['spoiler']== False, 0, 1)

To also have sentence-wise labels and review text without labels, we define and apply the following functions:

In [19]:
#Get only the labels 0 and 1 from the review
def get_labels(x):
    return [label for label, text in x]

#Get only the text from the review
def get_text(x):
    return [text for label, text in x]

In [20]:
#Apply the function to the data
train['sentence_labels'] = train.review.apply(lambda x: get_labels(x))
train['review_texts'] = train.review.apply(lambda x: get_text(x))

In [21]:
train.review_texts = train.review_texts.astype('str')

We also want to delete special and digits characters from the review text and lower the text.

In [22]:
#import special characters, transform them to list and add digits
from string import punctuation
specials = list(punctuation)
specials.extend(['1', '2', '3', '4', '5', '6', '7', '8', '9', '0'])

In [23]:
train['raw_text'] = pd.Series('str')

In [ ]:
import re
for i in range(len(train)):
    train['raw_text'][i] = re.sub('[^a-zA-Z " "]', '', train['review_texts'].copy()[i])

/opt/anaconda3/envs/nf/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
train.head()

We finally compute two features with the length (word-wise) of each review since one can hypothesize that longer reviews are more likely to contain spoilers than shorter ones.

In [ ]:
train['review_len'] = train.review_texts.str.split(' ').map(len)

#### Missing values

In [ ]:
#Denote missing values as np.nan instead of ''.  
train.replace('', np.nan, inplace = True)

In [ ]:
# We want to see how many missing values are in every column (as relative frequencies): 
for col in train.columns:
    pct_missing = np.mean(train[col].isna())
    print('{} - {}%'.format(col, round(pct_missing*100, 2)))

Publication year, month and day also contain missing values. We drop month and day features as well as the rows with missing values for the publication year.
Missing values for num_pages are also dropped.

This means a reduction of the train set by 12.3%. Since all features containing NaNs are not of major importance, the reduced dataset is stored separately and will be used only when needed.

In [ ]:
#Copy the original train set
train_red = train.copy()

In [ ]:
# Drop the columns and rows not needed in the copied dataframe
train_red.drop(columns = ['publication_month', 'publication_day'], axis = 1, inplace = True)
train_red.dropna(subset = ['publication_year', 'num_pages'], inplace = True)

#### Duplicates

Any duplicates are dropped.

In [ ]:
train[train.duplicated()]
train_red[train_red.duplicated()]

In [ ]:
train.drop_duplicates(inplace = True)
train_red.drop_duplicates(inplace = True)

#### Outliers

We explore boxplots of numeric features for outlier detection.

In [ ]:
fig, axes = pyplot.subplots(ncols=4, figsize=(12, 5), sharey = False)
train[['rating', 'publication_year', 'average_rating', 'ratings_count', 'num_pages', 'review_len']].boxplot(return_type='axes', ax=axes)

Generally, outliers play only a minor role for our project: our main subject is the classification of reviews with regard to spoilers, which might be modulated by other features (we will learn about that in the EDA) but ... Therefore, outliers will not be removed from the dataframe but we will account for them in the feature standardization. 

In [ ]:
#Safe the reduced dataframe as HDF5
train_red.to_hdf('data/train_reduced.h5', key = 'red')

#Save the not reduced dataframe as HDF5
train_.to_hdf('data/train_data.h5', key = 'train')

#### Rescaling of numeric variables

Data are rescaled using the RobustScaler.
The centering and scaling statistics of this scaler are based on percentiles and are therefore not influenced by a few number of very large marginal outliers. Consequently, the resulting range of the transformed feature values is larger than for the previous scalers and, more importantly, are approximately similar.

In [ ]:
train.columns

In [ ]:
#Rescale data
from sklearn.preprocessing import RobustScaler
from sklearn_pandas import DataFrameMapper

#Select numerica variables
train_num = train.copy().select_dtypes('number')

#Robust Scaler
mapper = DataFrameMapper([(train_num, RobustScaler())])
scaled_features = mapper.fit_transform(train_num.copy(), 4)
train_num_scaled = pd.DataFrame(scaled_features, index=train_num.index, columns=train_num.columns)

In [ ]:
#Rescale reduced data
train_red_num = train_red.copy().select_dtypes('number')
mapper = DataFrameMapper([(train_red_num, RobustScaler())])
scaled_features = mapper.fit_transform(train_red_num.copy(), 4)
train_num_red_scaled = pd.DataFrame(scaled_features, index=train_red_num.index, columns=train_red_num.columns)

In [ ]:
#Isolate the non-numeric variables
train_obj = train.copy().selectdtypes('object', 'datetime')
train_red_obj = train_red.copy().selectdtypes('object', 'datetime')

In [ ]:
# Concatenate train_scaled = pd.concat([train_num_scaled, train_obj],axis = 1)
train_red_scaled = pd.concat([train_num_red_scaled, train_obj],axis = 1)

In [ ]:
#Safe the reduced dataframe as HDF5
train_red_scaled.to_hdf('data/train_reduced_scaled.h5', key = 'red')

#Save the not reduced dataframe as HDF5
train_scaled.to_hdf('data/train_data_scaled.h5', key = 'train')